# Pipeline ETL - Base de Datos Fórmula 1
Este notebook carga los datos de los CSVs de Kaggle y los datos scrapeados en MySQL.

## 1. Conexión a MySQL

In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
# Configuración de conexión - CAMBIA ESTOS VALORES
DB_USER = "root"
DB_PASSWORD = "tu_password"  # <-- Cambia esto
DB_HOST = "127.0.0.1"
DB_NAME = "f1_db"

In [ ]:
# Conectar a MySQL (sin especificar base de datos)
con = mysql.connector.connect(user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
cursor = con.cursor()
print("Conexión exitosa!")

## 2. Crear Base de Datos y Tablas

In [ ]:
# Eliminar BD si existe y crearla de nuevo
cursor.execute("DROP DATABASE IF EXISTS f1_db;")
cursor.execute("CREATE DATABASE f1_db;")
cursor.execute("USE f1_db;")
print("Base de datos f1_db creada!")

### Tabla PILOTO

In [ ]:
sql = """
CREATE TABLE PILOTO (
    driverId INT PRIMARY KEY,
    driverRef VARCHAR(50),
    driverNumber INT,
    driverCode VARCHAR(3),
    forename VARCHAR(50),
    surname VARCHAR(50),
    dob DATE,
    nationality VARCHAR(50),
    url VARCHAR(255)
);
"""
cursor.execute(sql)
print("Tabla PILOTO creada!")

### Tabla EQUIPO

In [ ]:
sql = """
CREATE TABLE EQUIPO (
    constructorId INT PRIMARY KEY,
    constructorRef VARCHAR(50),
    constructorName VARCHAR(70),
    nationality VARCHAR(50),
    url VARCHAR(255)
);
"""
cursor.execute(sql)
print("Tabla EQUIPO creada!")

### Tabla CIRCUITO

In [ ]:
sql = """
CREATE TABLE CIRCUITO (
    circuitId INT PRIMARY KEY,
    circuitRef VARCHAR(50),
    circuitName VARCHAR(70),
    location VARCHAR(50),
    country VARCHAR(50),
    lat DECIMAL(10, 6),
    lng DECIMAL(10, 6),
    alt INT,
    url VARCHAR(255)
);
"""
cursor.execute(sql)
print("Tabla CIRCUITO creada!")

### Tabla CARRERA

In [ ]:
sql = """
CREATE TABLE CARRERA (
    raceId INT PRIMARY KEY,
    raceYear INT,
    raceRound INT,
    circuitId INT,
    raceName VARCHAR(100),
    raceDate DATE,
    raceTime TIME,
    url VARCHAR(255),
    FOREIGN KEY (circuitId) REFERENCES CIRCUITO(circuitId)
);
"""
cursor.execute(sql)
print("Tabla CARRERA creada!")

### Tabla RESULTADO

In [ ]:
sql = """
CREATE TABLE RESULTADO (
    resultId INT PRIMARY KEY,
    raceId INT,
    driverId INT,
    constructorId INT,
    grid INT,
    positionResult INT,
    points DECIMAL(6,2),
    laps INT,
    totalTime VARCHAR(50),
    FOREIGN KEY (raceId) REFERENCES CARRERA(raceId),
    FOREIGN KEY (driverId) REFERENCES PILOTO(driverId),
    FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId)
);
"""
cursor.execute(sql)
print("Tabla RESULTADO creada!")

### Tabla CLASIFICACION

In [ ]:
sql = """
CREATE TABLE CLASIFICACION (
    qualifyingId INT PRIMARY KEY,
    raceId INT,
    driverId INT,
    constructorId INT,
    qualifyingPosition INT,
    q1 VARCHAR(50),
    q2 VARCHAR(50),
    q3 VARCHAR(50),
    FOREIGN KEY (raceId) REFERENCES CARRERA(raceId),
    FOREIGN KEY (driverId) REFERENCES PILOTO(driverId),
    FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId)
);
"""
cursor.execute(sql)
print("Tabla CLASIFICACION creada!")

### Tabla INFO_CIRCUITO (datos scrapeados)

In [ ]:
sql = """
CREATE TABLE INFO_CIRCUITO (
    circuitId INT PRIMARY KEY,
    capacity VARCHAR(50),
    opened_year INT,
    length VARCHAR(50),
    turns INT,
    FOREIGN KEY (circuitId) REFERENCES CIRCUITO(circuitId)
);
"""
cursor.execute(sql)
print("Tabla INFO_CIRCUITO creada!")

### Tabla INFO_EQUIPO (datos scrapeados)

In [ ]:
sql = """
CREATE TABLE INFO_EQUIPO (
    constructorId INT PRIMARY KEY,
    championships INT,
    race_wins INT,
    pole_positions INT,
    fastest_laps INT,
    FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId)
);
"""
cursor.execute(sql)
print("Tabla INFO_EQUIPO creada!")

In [ ]:
# Verificar que todas las tablas se crearon
cursor.execute("SHOW TABLES;")
print("Tablas creadas:")
for tabla in cursor.fetchall():
    print(f"  - {tabla[0]}")

## 3. Funciones Auxiliares

In [ ]:
def mostrar_sql(sentencia_sql):
    """Ejecuta una sentencia SQL y muestra los resultados"""
    cursor.execute(sentencia_sql)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

In [ ]:
def limpiar_valor(valor):
    """Convierte valores problemáticos a None para MySQL"""
    if pd.isna(valor):
        return None
    if valor == "\\N" or valor == "":
        return None
    return valor

In [ ]:
def limpiar_dataframe(df):
    """Aplica limpieza a todo el DataFrame"""
    # Reemplazar \N por None
    df = df.replace("\\N", None)
    df = df.replace("", None)
    # Reemplazar NaN por None
    df = df.where(pd.notnull(df), None)
    return df

## 4. Cargar Datos desde CSVs

**Orden de carga importante** (por las foreign keys):
1. PILOTO, EQUIPO, CIRCUITO (tablas sin dependencias)
2. CARRERA (depende de CIRCUITO)
3. RESULTADO, CLASIFICACION (dependen de varias tablas)
4. INFO_CIRCUITO, INFO_EQUIPO (datos scrapeados)

### 4.1 Cargar PILOTO

In [ ]:
# Leer CSV
df_drivers = pd.read_csv("data/raw/drivers.csv")
print(f"Filas en drivers.csv: {len(df_drivers)}")
df_drivers.head()

In [ ]:
# Limpiar datos
df_drivers = limpiar_dataframe(df_drivers)

# Insertar datos
sql_insert = """
INSERT INTO PILOTO (driverId, driverRef, driverNumber, driverCode, forename, surname, dob, nationality, url)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

contador = 0
for index, row in df_drivers.iterrows():
    valores = (
        row["driverId"],
        row["driverRef"],
        row["number"],
        row["code"],
        row["forename"],
        row["surname"],
        row["dob"],
        row["nationality"],
        row["url"]
    )
    cursor.execute(sql_insert, valores)
    contador += 1

con.commit()
print(f"Insertados {contador} pilotos en PILOTO")

In [ ]:
# Verificar carga
cursor.execute("SELECT COUNT(*) FROM PILOTO;")
print(f"Total pilotos en BD: {cursor.fetchone()[0]}")

### 4.2 Cargar EQUIPO

In [ ]:
# Leer CSV
df_constructors = pd.read_csv("data/raw/constructors.csv")
print(f"Filas en constructors.csv: {len(df_constructors)}")
df_constructors.head()

In [ ]:
# Limpiar datos
df_constructors = limpiar_dataframe(df_constructors)

# Insertar datos
sql_insert = """
INSERT INTO EQUIPO (constructorId, constructorRef, constructorName, nationality, url)
VALUES (%s, %s, %s, %s, %s)
"""

contador = 0
for index, row in df_constructors.iterrows():
    valores = (
        row["constructorId"],
        row["constructorRef"],
        row["name"],
        row["nationality"],
        row["url"]
    )
    cursor.execute(sql_insert, valores)
    contador += 1

con.commit()
print(f"Insertados {contador} equipos en EQUIPO")

In [ ]:
# Verificar carga
cursor.execute("SELECT COUNT(*) FROM EQUIPO;")
print(f"Total equipos en BD: {cursor.fetchone()[0]}")

### 4.3 Cargar CIRCUITO

In [ ]:
# Leer CSV
df_circuits = pd.read_csv("data/raw/circuits.csv")
print(f"Filas en circuits.csv: {len(df_circuits)}")
df_circuits.head()

In [ ]:
# Limpiar datos
df_circuits = limpiar_dataframe(df_circuits)

# Insertar datos
sql_insert = """
INSERT INTO CIRCUITO (circuitId, circuitRef, circuitName, location, country, lat, lng, alt, url)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

contador = 0
for index, row in df_circuits.iterrows():
    valores = (
        row["circuitId"],
        row["circuitRef"],
        row["name"],
        row["location"],
        row["country"],
        row["lat"],
        row["lng"],
        row["alt"],
        row["url"]
    )
    cursor.execute(sql_insert, valores)
    contador += 1

con.commit()
print(f"Insertados {contador} circuitos en CIRCUITO")

In [ ]:
# Verificar carga
cursor.execute("SELECT COUNT(*) FROM CIRCUITO;")
print(f"Total circuitos en BD: {cursor.fetchone()[0]}")

### 4.4 Cargar CARRERA

In [ ]:
# Leer CSV
df_races = pd.read_csv("data/raw/races.csv")
print(f"Filas en races.csv: {len(df_races)}")
df_races.head()

In [ ]:
# Limpiar datos
df_races = limpiar_dataframe(df_races)

# Insertar datos
sql_insert = """
INSERT INTO CARRERA (raceId, raceYear, raceRound, circuitId, raceName, raceDate, raceTime, url)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

contador = 0
for index, row in df_races.iterrows():
    valores = (
        row["raceId"],
        row["year"],
        row["round"],
        row["circuitId"],
        row["name"],
        row["date"],
        row["time"],
        row["url"]
    )
    cursor.execute(sql_insert, valores)
    contador += 1

con.commit()
print(f"Insertadas {contador} carreras en CARRERA")

In [ ]:
# Verificar carga
cursor.execute("SELECT COUNT(*) FROM CARRERA;")
print(f"Total carreras en BD: {cursor.fetchone()[0]}")

### 4.5 Cargar RESULTADO

In [ ]:
# Leer CSV
df_results = pd.read_csv("data/raw/results.csv")
print(f"Filas en results.csv: {len(df_results)}")
df_results.head()

In [ ]:
# Limpiar datos
df_results = limpiar_dataframe(df_results)

# Insertar datos
sql_insert = """
INSERT INTO RESULTADO (resultId, raceId, driverId, constructorId, grid, positionResult, points, laps, totalTime)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

contador = 0
for index, row in df_results.iterrows():
    valores = (
        row["resultId"],
        row["raceId"],
        row["driverId"],
        row["constructorId"],
        row["grid"],
        row["position"],
        row["points"],
        row["laps"],
        row["time"]
    )
    cursor.execute(sql_insert, valores)
    contador += 1

con.commit()
print(f"Insertados {contador} resultados en RESULTADO")

In [ ]:
# Verificar carga
cursor.execute("SELECT COUNT(*) FROM RESULTADO;")
print(f"Total resultados en BD: {cursor.fetchone()[0]}")

### 4.6 Cargar CLASIFICACION

In [ ]:
# Leer CSV
df_qualifying = pd.read_csv("data/raw/qualifying.csv")
print(f"Filas en qualifying.csv: {len(df_qualifying)}")
df_qualifying.head()

In [ ]:
# Limpiar datos
df_qualifying = limpiar_dataframe(df_qualifying)

# Insertar datos
sql_insert = """
INSERT INTO CLASIFICACION (qualifyingId, raceId, driverId, constructorId, qualifyingPosition, q1, q2, q3)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

contador = 0
for index, row in df_qualifying.iterrows():
    valores = (
        row["qualifyId"],
        row["raceId"],
        row["driverId"],
        row["constructorId"],
        row["position"],
        row["q1"],
        row["q2"],
        row["q3"]
    )
    cursor.execute(sql_insert, valores)
    contador += 1

con.commit()
print(f"Insertadas {contador} clasificaciones en CLASIFICACION")

In [ ]:
# Verificar carga
cursor.execute("SELECT COUNT(*) FROM CLASIFICACION;")
print(f"Total clasificaciones en BD: {cursor.fetchone()[0]}")

### 4.7 Cargar INFO_CIRCUITO (datos scrapeados)

In [ ]:
# Leer CSV scrapeado
df_info_circuitos = pd.read_csv("data/scraped/Final_Scraped_Circuitos.csv")
print(f"Filas en Final_Scraped_Circuitos.csv: {len(df_info_circuitos)}")
df_info_circuitos.head()

In [ ]:
# Limpiar datos
df_info_circuitos = limpiar_dataframe(df_info_circuitos)

# Insertar datos
sql_insert = """
INSERT INTO INFO_CIRCUITO (circuitId, capacity, opened_year, length, turns)
VALUES (%s, %s, %s, %s, %s)
"""

contador = 0
for index, row in df_info_circuitos.iterrows():
    valores = (
        row["circuitId"],
        row["capacity"],
        row["opened_year"],
        row["length"],
        row["turns"]
    )
    cursor.execute(sql_insert, valores)
    contador += 1

con.commit()
print(f"Insertados {contador} registros en INFO_CIRCUITO")

In [ ]:
# Verificar carga
cursor.execute("SELECT COUNT(*) FROM INFO_CIRCUITO;")
print(f"Total info circuitos en BD: {cursor.fetchone()[0]}")

### 4.8 Cargar INFO_EQUIPO (datos scrapeados)

In [ ]:
# Leer CSV scrapeado
df_info_equipos = pd.read_csv("data/scraped/Final_Scraped_Equipos.csv")
print(f"Filas en Final_Scraped_Equipos.csv: {len(df_info_equipos)}")
df_info_equipos.head()

In [ ]:
# Limpiar datos
df_info_equipos = limpiar_dataframe(df_info_equipos)

# Solo insertar equipos que existen en EQUIPO (por la FK)
cursor.execute("SELECT constructorId FROM EQUIPO;")
ids_validos = set([row[0] for row in cursor.fetchall()])

# Insertar datos
sql_insert = """
INSERT INTO INFO_EQUIPO (constructorId, championships, race_wins, pole_positions, fastest_laps)
VALUES (%s, %s, %s, %s, %s)
"""

contador = 0
saltados = 0
for index, row in df_info_equipos.iterrows():
    # Verificar que el constructorId existe en EQUIPO
    if row["constructorId"] not in ids_validos:
        saltados += 1
        continue
    
    valores = (
        row["constructorId"],
        row["championships"],
        row["race_wins"],
        row["pole_positions"],
        row["fastest_laps"]
    )
    cursor.execute(sql_insert, valores)
    contador += 1

con.commit()
print(f"Insertados {contador} registros en INFO_EQUIPO")
print(f"Saltados {saltados} registros (sin equipo correspondiente)")

In [ ]:
# Verificar carga
cursor.execute("SELECT COUNT(*) FROM INFO_EQUIPO;")
print(f"Total info equipos en BD: {cursor.fetchone()[0]}")

## 5. Verificación Final

In [ ]:
# Resumen de todas las tablas
print("=" * 40)
print("RESUMEN DE CARGA ETL")
print("=" * 40)

tablas = ["PILOTO", "EQUIPO", "CIRCUITO", "CARRERA", "RESULTADO", "CLASIFICACION", "INFO_CIRCUITO", "INFO_EQUIPO"]

for tabla in tablas:
    cursor.execute(f"SELECT COUNT(*) FROM {tabla};")
    count = cursor.fetchone()[0]
    print(f"{tabla}: {count} registros")

print("=" * 40)
print("ETL completado!")

In [ ]:
# Ejemplo: Ver algunos pilotos
print("Primeros 5 pilotos:")
mostrar_sql("SELECT driverId, forename, surname, nationality FROM PILOTO LIMIT 5;")

In [ ]:
# Ejemplo: JOIN entre tablas (verificar relaciones)
print("Últimas 5 carreras con su circuito:")
sql = """
SELECT c.raceName, c.raceYear, ci.circuitName, ci.country
FROM CARRERA c
INNER JOIN CIRCUITO ci ON c.circuitId = ci.circuitId
ORDER BY c.raceDate DESC
LIMIT 5;
"""
mostrar_sql(sql)

In [ ]:
# Cerrar conexión
cursor.close()
con.close()
print("Conexión cerrada!")